In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# from sklearn.metrics import confusion_matrix, accuracy_score


# from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
# import time
# import scipy.cluster.hierarchy as sch
# from sklearn.neighbors import NearestNeighbors


# eye candy plots
plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-light.mplstyle')
# source https://github.com/dhaitz/matplotlib-stylesheets

# from sklearn.model_selection import train_test_split, cross_val_score
# from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# from sklearn.linear_model import SGDClassifier
# from sklearn.neighbors import KNeighborsClassifier

In [ ]:
df = pd.read_csv("../../data/processed/aggregated_pueblos.csv")

In [ ]:
df.province.unique()

In [ ]:
df = df.drop(df[df['province'].isin(["Illes Balears", "Santa Cruz de Tenerife"])].index)

In [ ]:
sns.histplot(df, x="total_population", bins=30)


In [ ]:
# Plotting the boxplot
sns.boxplot(df["total_population"], color="teal")

In [ ]:
# Define function to print the whiskers of a boxplot

def calculate_whiskers(df, column):
    """
    Calculate the lower and upper whiskers for a specified column in a DataFrame.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the data.
        column (str): The column name to calculate whiskers for.

    Returns:
        tuple: (lower_whisker, upper_whisker)
    """
    # Get descriptive statistics for the column
    stats = df[column].describe()
    Q1 = stats['25%']
    Q3 = stats['75%']
    IQR = Q3 - Q1

    # Calculate the lower whisker
    lower_whisker = max(
        df[column][df[column] >= (Q1 - 1.5 * IQR)].min(), 
        df[column].min()
    )
    
    # Calculate the upper whisker
    upper_whisker = min(
        df[column][df[column] <= (Q3 + 1.5 * IQR)].max(), 
        df[column].max()
    )
    
    return lower_whisker, upper_whisker

In [ ]:
lower, upper = calculate_whiskers(df, "total_population")
print("Upper Whisker total population", upper)

In [ ]:
df.info()

In [ ]:
df_cities = df.query("total_population > 20000")

In [ ]:
title = "Cities in Spain"

lats = df_cities.latitude
lons = df_cities.longitude

fig = px.scatter_map(df_cities, 
                     lat=lats, 
                     lon=lons,
                     hover_data=["municipality"],
                     color_continuous_scale=px.colors.carto.Aggrnyl,
                     zoom=5,
                     size_max=50  # Increase max size of markers
                     )

# Adjust the size reference to make small points more visible
fig.update_traces(marker=dict(sizeref=1000))  # Decrease this value to make points larger

fig.update_geos(fitbounds="locations")
fig.update_layout(height=1000, width=1000)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":50,"t":50,"l":50,"b":50})
fig.update_layout(
    coloraxis_colorbar=dict(title='asdf')
)
fig.update_layout(title="Major in Spain ")

fig.show()

In [ ]:
df_cities.to_csv("../../data/processed/split_cities.csv")

In [ ]:
df_cities

In [ ]:
# df = df.query("total_population <= 6000")

In [ ]:
sns.histplot(df, x="total_population", bins=30)

In [ ]:
# Plotting the boxplot
sns.boxplot(df["total_population"], color="teal")

In [ ]:
df


In [ ]:
df.columns

In [ ]:
# Create the figure and first y-axis
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot the bar chart for total cost
sns.countplot(data=df, x= "province",order=df["province"].value_counts().index, ax=ax1, color="teal", alpha = 0.8)
plt.xticks(rotation=90)


In [ ]:
# Group by province and sum the total population
df_grouped = df.groupby("province", as_index=False)["total_population"].sum()

# Sort the grouped DataFrame by total_population in descending order
df_sorted = df_grouped.sort_values(by="total_population", ascending=False)

# Create the figure and first y-axis
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot the bar chart with ordered categories
sns.barplot(x=df_sorted.province, y=df_sorted.total_population, ax=ax1, color="teal", order=df_sorted.province, alpha = 0.8)

ax1.set_ylabel("Population", fontsize=12)
ax1.set_xlabel("Province", fontsize=12)
ax1.tick_params(axis='y')
ax1.tick_params(axis='x', rotation=90)
ax1.set_title("Total Population living in small towns per province", fontsize=14, pad=15)

plt.show()



In [ ]:
# Group by province and calculate the average population per municipality
df_grouped = df.groupby("province", as_index=False)["total_population"].mean()

# Sort the grouped DataFrame by average total_population in descending order
df_sorted = df_grouped.sort_values(by="total_population", ascending=False)

# Create the figure and first y-axis
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot the bar chart with ordered categories
sns.barplot(x=df_sorted.province, y=df_sorted.total_population, ax=ax1, color="teal", order=df_sorted.province, alpha = 0.8)

ax1.set_ylabel("Average Population per Municipality", fontsize=12)
ax1.set_xlabel("Province", fontsize=12)
ax1.tick_params(axis='y')
ax1.tick_params(axis='x', rotation=90)
ax1.set_title("Average Population per Municipality in Each Province", fontsize=14, pad=15)

plt.show()


In [ ]:
# Define function to categorize connectivity levels
def categorize_population(size):
    if size >= 3000:
        return "Big"
    elif size >= 500:
        return "Mid"
    elif size >= 100:
        return "Small"
    else:
        return "Very Small"

# Apply the function to create a new column
df["town_size"] = df["total_population"].apply(categorize_population)

In [ ]:
df

In [ ]:
order = ["Very Small", "Small", "Mid", "Big"]

In [ ]:
# Create the figure and first y-axis
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot the bar chart for total cost
sns.countplot(data=df, x= "town_size",order=order, ax=ax1, color="teal", alpha = 0.8)

What questions would be useful to answer with this EDA?

In [ ]:
# Group by province and calculate the average population per municipality
df_connectivity = df[['vdsl_30mbps', 'fixed_wireless', 'ftth', 'hfc', 'reception_30mbps','reception_100mbps', 'reception_1gbps', '4g', '5g', '5g_3,5ghz']]

# Plot the bar chart with ordered categories
sns.pairplot(df_connectivity)


Meanings of the Connectivity Indicators:

Each of these refers to a different type of broadband or mobile network connectivity in a town. Here's what they mean:

vdsl_30mbps → Availability of VDSL (Very-high-bit-rate Digital Subscriber Line) at 30 Mbps. A faster version of DSL that uses telephone lines.
fixed_wireless → Percentage of coverage by Fixed Wireless Access (FWA), which provides broadband via radio signals instead of cables.
ftth → Fiber-to-the-Home (FTTH) availability, meaning direct fiber optic connections to residences (highest speed and reliability).
hfc → Hybrid Fiber-Coaxial (HFC) availability, a mix of fiber optics and coaxial cable (used in cable internet services).
reception_30mbps → Percentage of the area that can receive at least 30 Mbps (regardless of technology).
reception_100mbps → Percentage of the area that can receive at least 100 Mbps.
reception_1gbps → Percentage of the area that can receive at least 1 Gbps (1000 Mbps).
4g → Coverage of 4G mobile network.
5g → Coverage of 5G mobile network (general).
5g_3,5ghz → Coverage of 5G at 3.5 GHz, a specific frequency band that offers higher speeds and lower latency.

The 3 Most Important Metrics to Define Connectivity in a Town:

ftth (Fiber-to-the-Home)
Why? It's the gold standard for broadband, offering the fastest speeds, low latency, and high reliability.
Key Impact: Towns with high FTTH coverage have superior internet quality.

reception_100mbps or reception_1gbps
Why? This metric shows how much of the town has access to fast internet speeds (regardless of the technology).
Key Impact: Ensures people and businesses can get modern broadband speeds.

5g or 5g_3,5ghz
Why? 5G is essential for mobile and future-proof connectivity (low latency, high-speed mobile broadband).
Key Impact: Towns with strong 5G networks can support smart city applications, IoT, and next-gen mobile services.

In [ ]:
# Group by province and calculate the average population per municipality
df_connectivity = df[[ 'ftth','reception_100mbps', '4g']]

# Plot the bar chart with ordered categories
sns.pairplot(df_connectivity)


### **Adjusted Weight Distribution**
| **Factor**            | **New Weight (%)** | **Reasoning** |
|-----------------------|-------------------|--------------|
| `ftth`               | **50%**            | Fiber is the most important for stable, high-speed connectivity. |
| `reception_100mbps`  | **35%**            | Ensures fast broadband availability, even if not fiber. |
| `4g`                 | **15%**            | Still essential for mobile broadband, but not the primary factor. |

In [ ]:
# Define new weights
weights = {
    'ftth': 0.5,
    'reception_100mbps': 0.35,
    '4g': 0.15
}

# Compute the adjusted Connectivity Score
df["connectivity_score"] = (
    df["ftth"] * weights["ftth"] +
    df["reception_100mbps"] * weights["reception_100mbps"] +
    df["4g"] * weights["4g"]
)


### **Connectivity Score Categories**
| **Score Range**  | **Category**          | **Description** |
|------------------|----------------------|----------------|
| **80 - 100**     | **Excellent**         | Strong fiber coverage and high-speed internet. |
| **60 - 79**      | **Good**              | Decent broadband with fiber or high-speed non-fiber options. |
| **40 - 59**      | **Moderate**          | Some high-speed coverage, but fiber may be limited. |
| **20 - 39**      | **Weak**              | Basic connectivity with limited high-speed access. |
| **0 - 19**       | **Poor**              | Very poor or no access to high-speed internet. |

In [ ]:
# Define function to categorize connectivity levels
def categorize_connectivity(score):
    if score >= 0.8:
        return "Excellent"
    elif score >= 0.6:
        return "Good"
    elif score >= 0.4:
        return "Moderate"
    elif score >= 0.2:
        return "Weak"
    else:
        return "Poor"

# Apply the function to create a new column
df["connectivity_category"] = df["connectivity_score"].apply(categorize_connectivity)


In [ ]:
df

In [ ]:
sns.histplot(df, x="connectivity_category", bins=30, color = "teal", alpha = 0.8)

In [ ]:
df.drop(columns=["ftth", "hfc", "reception_30mbps", "reception_100mbps", "reception_1gbps", "4g", "5g", "5g_3,5ghz", 'vdsl_30mbps','fixed_wireless'], axis=1, inplace=True)

In [ ]:
df[['n_industry', 'n_construction',
       'n_info_communications', 'n_financial_insurance', 'n_real_estate',
       'n_professional_technical', 'n_eduation_health_social', 'n_other']].describe()

In [ ]:
title = "Villages in Spain"

lats = df.latitude
lons = df.longitude

fig = px.scatter_map(df, 
                     lat=lats, 
                     lon=lons,
                     hover_data=["municipality"],
                     color_continuous_scale=px.colors.carto.Aggrnyl,
                     zoom=5,
                     size_max=50  # Increase max size of markers
                     )
# Adjust the size reference to make small points more visible
fig.update_traces(marker=dict(sizeref=1000))  # Decrease this value to make points larger

fig.update_geos(fitbounds="locations")
fig.update_layout(height=1000, width=1000)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":50,"t":50,"l":50,"b":50})
fig.update_layout(
    coloraxis_colorbar=dict(title='asdf')
)
fig.update_layout(title="Major in Spain ")


fig.show()

In [ ]:
df[['latitude', 'longitude']].describe()

In [ ]:
bins = {'lat': np.arange(df['latitude'].min(), df['latitude'].max(), 0.5), 'lon': np.arange(df['longitude'].min(), df['longitude'].max(), 0.5)}

1. make bins of lat, long
2. sort pueplos into the bins
3rd -> for each pueblo: have a look at 1 lower and 1 higer (lat, long)

In [ ]:
df = df.copy()

df['lat_bin'] = pd.cut(df['latitude'], bins['lat'], labels=False, include_lowest=True)
df['lon_bin'] = pd.cut(df['longitude'], bins['lon'], labels=False, include_lowest=True)
df['lat_bin'] = df['lat_bin'].fillna(-1).astype(int)
df['lon_bin'] = df['lon_bin'].fillna(-1).astype(int)
df


In [ ]:
from geopy.distance import geodesic
lon_max = df['lon_bin'].max()
lat_max = df['lat_bin'].max()

def get_towns_in_vicinity(row: pd.Series, distance):
    cmuns = []
    # Properly handle edge bins and include all surrounding quadrants
    for idx in range(max(0, row['lon_bin'] - 1), min(lon_max + 1, row['lon_bin'] + 2)):
        for idy in range(max(0, row['lat_bin'] - 1), min(lat_max + 1, row['lat_bin'] + 2)):
            cmuns.extend(df.loc[(df['lat_bin'] == idy) & (df['lon_bin'] == idx), 'cmun'].tolist())
    # Get the origin coordinates
    origin = (row['latitude'], row['longitude'])
    # Select rows for comparison
    new_df = df[df["cmun"].isin(cmuns)].copy()
    # Remove the current town if necessary
    new_df = new_df[new_df['cmun'] != row['cmun']]
    # Calculate distances
    new_df["distance"] = [geodesic(origin, (lat, lon)).kilometers for lat, lon in zip(new_df["latitude"], new_df["longitude"])]
    new_df = new_df[new_df["distance"] < distance]
    return new_df['cmun'].to_list()


In [ ]:
# Apply the function
df['towns_in_vicinity'] = df.apply(lambda x: get_towns_in_vicinity(x, 40), axis=1)

In [ ]:
df_economy = df[['n_industry', 'n_construction',
       'n_info_communications', 'n_financial_insurance', 'n_real_estate',
       'n_professional_technical', 'n_eduation_health_social', 'n_other']]

# Define weights for each economic indicator (summing to 1)
economy_weights = {
    'n_industry': 0.2,
    'n_construction': 0.15,
    'n_info_communications': 0.1,
    'n_financial_insurance': 0.1,
    'n_real_estate': 0.1,
    'n_professional_technical': 0.15,
    'n_eduation_health_social': 0.1,
    'n_other': 0.1
}

# Calculate a weighted economy score using the defined weights
df["economy_score"] = df_economy.multiply(pd.Series(economy_weights)).sum(axis=1)


In [ ]:
def combine_economy_scores(row):
    base_score = row["economy_score"]
    vicinity = row["towns_in_vicinity"]
    if vicinity:
        neighbor_scores = df.loc[df["cmun"].isin(vicinity), "economy_score"]
        if not neighbor_scores.empty:
            avg_neighbor_score = neighbor_scores.mean()
        else:
            avg_neighbor_score = 0
        return (base_score + avg_neighbor_score) / 2
    else:
        return base_score

df["economy_score_area"] = df.apply(combine_economy_scores, axis=1)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df

In [ ]:
df.columns

In [ ]:
sns.histplot(df.latitude)

In [ ]:
sns.histplot(df.longitude)

In [ ]:
df["towns_in_vicinity"] = df["towns_in_vicinity"].apply(
    lambda x: ", ".join(map(str, x)) if isinstance(x, list) else x
)

In [ ]:
type(df.iloc[0]['towns_in_vicinity'])

df.query("municipality == 'Ferreries'")

In [ ]:
df.to_csv("../../data/processed/2_aggregated_pueblos.csv", index=False)

In [ ]:
df.towns_in_vicinity.isnull().sum()

In [ ]:
df